# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-05 21:28:43] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36054, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=23937

[2025-08-05 21:28:43] Using default HuggingFace chat template with detected content format: string


[2025-08-05 21:28:52] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-05 21:28:52] Init torch distributed begin.


[2025-08-05 21:28:52] Init torch distributed ends. mem usage=0.00 GB


[2025-08-05 21:28:53] Load weight begin. avail mem=62.17 GB


...

[2025-08-05 21:28:56] Init torch distributed begin.
[2025-08-05 21:28:56] Init torch distributed ends. mem usage=0.00 GB
[2025-08-05 21:28:56] Load weight begin. avail mem=38.65 GB
[2025-08-05 21:28:57] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.15s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.15s/it]

[2025-08-05 21:28:58] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-08-05 21:28:58] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-05 21:28:58] Memory pool end. avail mem=37.40 GB


[2025-08-05 21:28:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.63 GB
[2025-08-05 21:28:58] INFO:     Started server process [965828]
[2025-08-05 21:28:58] INFO:     Waiting for application startup.
[2025-08-05 21:28:58] INFO:     Application startup complete.
[2025-08-05 21:28:58] INFO:     Uvicorn running on http://127.0.0.1:36054 (Press CTRL+C to quit)


[2025-08-05 21:28:58] INFO:     127.0.0.1:39330 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-05 21:28:59] INFO:     127.0.0.1:39344 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-05 21:28:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-05 21:30:03] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-05 21:30:32] INFO:     127.0.0.1:39348 - "POST /generate HTTP/1.1" 200 OK
[2025-08-05 21:30:32] The server is fired up and ready to roll!


[2025-08-05 21:30:35] INFO:     127.0.0.1:34606 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-05 21:30:42] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32158, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=8878772

[2025-08-05 21:30:42] Using default HuggingFace chat template with detected content format: string


[2025-08-05 21:30:49] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-05 21:30:49] Init torch distributed begin.


[2025-08-05 21:30:50] Init torch distributed ends. mem usage=0.00 GB


[2025-08-05 21:30:52] Load weight begin. avail mem=78.58 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.16s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.16s/it]

[2025-08-05 21:30:56] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=39.29 GB, mem usage=15.37 GB.
[2025-08-05 21:30:56] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-05 21:30:56] Memory pool end. avail mem=38.97 GB


[2025-08-05 21:30:57] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=39.16 GB
[2025-08-05 21:30:57] INFO:     Started server process [971849]
[2025-08-05 21:30:57] INFO:     Waiting for application startup.
[2025-08-05 21:30:57] INFO:     Application startup complete.
[2025-08-05 21:30:57] INFO:     Uvicorn running on http://127.0.0.1:32158 (Press CTRL+C to quit)


[2025-08-05 21:30:57] INFO:     127.0.0.1:42262 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-05 21:30:58] INFO:     127.0.0.1:42272 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-05 21:30:58] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:31:01] INFO:     127.0.0.1:42284 - "POST /generate HTTP/1.1" 200 OK
[2025-08-05 21:31:01] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-05 21:31:02] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:31:03] INFO:     127.0.0.1:42294 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-05 21:31:10] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=31597, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-05 21:31:11] Using default HuggingFace chat template with detected content format: string


[2025-08-05 21:31:20] Casting torch.bfloat16 to torch.float16.


[2025-08-05 21:31:20] Casting torch.bfloat16 to torch.float16.
[2025-08-05 21:31:20] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-05 21:31:20] Init torch distributed begin.


[2025-08-05 21:31:22] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-05 21:31:39] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=41.96 GB


[2025-08-05 21:31:40] INFO:     Started server process [973804]
[2025-08-05 21:31:40] INFO:     Waiting for application startup.
[2025-08-05 21:31:40] INFO:     Application startup complete.
[2025-08-05 21:31:40] INFO:     Uvicorn running on http://127.0.0.1:31597 (Press CTRL+C to quit)


[2025-08-05 21:31:41] INFO:     127.0.0.1:57930 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-05 21:31:41] INFO:     127.0.0.1:57936 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-05 21:31:41] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:31:44] INFO:     127.0.0.1:57942 - "POST /generate HTTP/1.1" 200 OK
[2025-08-05 21:31:44] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-05 21:31:46] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:31:47] INFO:     127.0.0.1:40028 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-05 21:31:54] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34902, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-05 21:31:55] Using default HuggingFace chat template with detected content format: string


[2025-08-05 21:32:02] Casting torch.bfloat16 to torch.float16.


[2025-08-05 21:32:02] Casting torch.bfloat16 to torch.float16.
[2025-08-05 21:32:03] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-05 21:32:03] Init torch distributed begin.


[2025-08-05 21:32:03] Init torch distributed ends. mem usage=0.00 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.04it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.04it/s]

[2025-08-05 21:32:21] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=40.83 GB, mem usage=2.15 GB.
[2025-08-05 21:32:21] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-08-05 21:32:21] Memory pool end. avail mem=40.75 GB


[2025-08-05 21:32:21] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=41.43 GB


[2025-08-05 21:32:22] INFO:     Started server process [974905]
[2025-08-05 21:32:22] INFO:     Waiting for application startup.
[2025-08-05 21:32:22] INFO:     Application startup complete.
[2025-08-05 21:32:22] INFO:     Uvicorn running on http://127.0.0.1:34902 (Press CTRL+C to quit)
[2025-08-05 21:32:22] INFO:     127.0.0.1:47320 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-05 21:32:23] INFO:     127.0.0.1:50734 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-05 21:32:23] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:32:25] INFO:     127.0.0.1:50742 - "POST /generate HTTP/1.1" 200 OK
[2025-08-05 21:32:25] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-05 21:32:27] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:32:28] INFO:     127.0.0.1:50746 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-05 21:32:36] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=39941, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1034546811, constrained_js

[2025-08-05 21:32:36] Using default HuggingFace chat template with detected content format: string


[2025-08-05 21:32:45] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-05 21:32:45] Init torch distributed begin.


[2025-08-05 21:32:46] Init torch distributed ends. mem usage=0.00 GB


[2025-08-05 21:32:47] Load weight begin. avail mem=78.58 GB


...

[2025-08-05 21:32:54] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=59.84 GB


[2025-08-05 21:32:55] INFO:     Started server process [976745]
[2025-08-05 21:32:55] INFO:     Waiting for application startup.
[2025-08-05 21:32:55] INFO:     Application startup complete.
[2025-08-05 21:32:55] INFO:     Uvicorn running on http://0.0.0.0:39941 (Press CTRL+C to quit)


[2025-08-05 21:32:55] INFO:     127.0.0.1:36790 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-05 21:32:56] INFO:     127.0.0.1:36796 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-05 21:32:56] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:32:57] INFO:     127.0.0.1:36798 - "POST /generate HTTP/1.1" 200 OK
[2025-08-05 21:32:57] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-05 21:33:00] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-05 21:33:00] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: False, gen throughput (token/s): 11.48, #queue-req: 0, 


[2025-08-05 21:33:01] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, accept len: 1.65, cuda graph: False, gen throughput (token/s): 98.11, #queue-req: 0, 


[2025-08-05 21:33:02] Decode batch. #running-req: 1, #token: 226, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 100.60, #queue-req: 0, 


[2025-08-05 21:33:03] Decode batch. #running-req: 1, #token: 294, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 90.41, #queue-req: 0, 


[2025-08-05 21:33:03] Decode batch. #running-req: 1, #token: 358, token usage: 0.02, accept len: 1.60, cuda graph: False, gen throughput (token/s): 83.73, #queue-req: 0, 


[2025-08-05 21:33:04] Decode batch. #running-req: 1, #token: 432, token usage: 0.02, accept len: 1.85, cuda graph: False, gen throughput (token/s): 102.94, #queue-req: 0, 


[2025-08-05 21:33:05] INFO:     127.0.0.1:36814 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).